In [5]:
import pandas as pd
import numpy as np
import re

In [ ]:
from movie_pkg.utils import *

In [ ]:
base = 'llm_checks' # change to fit however you have the data saved

In [15]:
import pandas as pd
import numpy as np
import re
from difflib import SequenceMatcher

SIM_THRESHOLD = 0.90  # we're using a similarity threshold because human coding can result in differences in spacing, 
#    capitalization, etc. which are not meaningful differences. We just care if the same utterance was mapped to the same ground-truth
#    hence the 90% similarity threshold

alignments = []

for subject in get_scan_subs():
    subject_scores = []
    # not all subjects watched both movies, so we have a utility to figure out which one's did and didn't
    for movie in get_subject_movies(subject):
        file1 = f"{base}/checked/subj-{subject}_{movie}_event_coded.tsv"
        file2 = f"{base}/scan_event_coding_final/{subject}/subj-{subject}_{movie}_event_coded.tsv"

        df1 = pd.read_csv(file1, sep="\t")
        df2 = pd.read_csv(file2, sep="\t")

        # match file length for comparison
        n = min(len(df1), len(df2))
        col1 = df1.loc[:n-1, 'detail_recall'].astype(str).fillna('').str.strip()
        col2 = df2.loc[:n-1, 'detail_recall'].astype(str).fillna('').str.strip()

        matches = []
        for a, b in zip(col1, col2):
            # remove punctuation and lowercase for comparison
            a_clean = re.sub(r'\W+', '', a.lower())
            b_clean = re.sub(r'\W+', '', b.lower())

            # match if both blank (no mapped event) OR sufficiently similar (same mapped event)
            if a_clean == '' and b_clean == '':
                matches.append(1)
            else:
                ratio = SequenceMatcher(None, a_clean, b_clean).ratio()
                matches.append(1 if ratio >= SIM_THRESHOLD else 0)

        subject_scores.append(float(np.mean(matches)) if matches else np.nan)

    alignments.append(np.nanmean(subject_scores) if subject_scores else np.nan)

alignments_df = pd.DataFrame({'subject': get_scan_subs(), 'alignment': alignments})
print(alignments_df)

      subject  alignment
0   temple056   0.722222
1   temple057   1.000000
2   temple058   0.856785
3   temple059   0.952072
4   temple060   0.928734
..        ...        ...
61  temple130   1.000000
62  temple131   1.000000
63  temple132   1.000000
64  temple135   1.000000
65  temple136   0.991803

[66 rows x 2 columns]


In [17]:
np.mean(alignments_df['alignment'])

0.942019236076613

### Human adjustments were minimal, with only 5.8% of codes requiring revision and 94.2% confirmed without change.